```{contents}
```
## Refine Chain

### What a Refine Chain Is

A **Refine Chain** is a LangChain abstraction used to **process large inputs incrementally**, where the model:

* Generates an initial answer from the **first chunk**
* **Refines** that answer step by step as **new chunks** are introduced

> Instead of summarizing chunks independently (MapReduce), the model **maintains and improves a single evolving answer**.

---

### Why Refine Chain Exists

MapReduce can:

* Lose global context
* Produce shallow summaries

Refine chain improves:

* Coherence
* Accuracy
* Context preservation

Especially useful when **order matters**.

---

### Conceptual Flow

```
Chunk 1 → Initial Answer
Chunk 2 → Refine Answer
Chunk 3 → Refine Answer
...
Final Answer
```

---

### How Refine Chain Works Internally

1. Process first document chunk
2. Generate an initial response
3. For each subsequent chunk:

   * Provide the previous answer
   * Ask the model to improve/refine it

---

### Refine vs MapReduce

| Aspect            | Refine Chain | MapReduce  |
| ----------------- | ------------ | ---------- |
| Parallelism       | ❌ Sequential | ✅ Parallel |
| Accuracy          | Higher       | Medium     |
| Latency           | Higher       | Lower      |
| Context retention | Strong       | Weak       |

---

### Basic Refine Chain Demonstration

#### Step 1: Initial Prompt



In [1]:

from langchain_core.prompts import PromptTemplate

initial_prompt = PromptTemplate.from_template(
    "Write a summary of the following text:\n{text}"
)


---

#### Step 2: Refine Prompt



In [2]:
refine_prompt = PromptTemplate.from_template(
    """
Existing summary:
{existing_answer}

New context:
{text}

Refine the summary using the new context.
"""
)



---

#### Step 3: Create Refine Chain



In [4]:
from langchain_classic.chains.summarize import load_summarize_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=initial_prompt,
    refine_prompt=refine_prompt
)




---

#### Step 4: Invoke the Chain



In [6]:
from langchain_core.documents import Document

# Create sample documents
documents = [
    Document(page_content="Artificial Intelligence is transforming industries worldwide. Machine learning algorithms enable computers to learn from data and improve their performance over time without explicit programming."),
    Document(page_content="Natural Language Processing allows machines to understand and generate human language. Applications include chatbots, translation services, and sentiment analysis."),
    Document(page_content="Computer vision enables machines to interpret and understand visual information from the world. It powers technologies like facial recognition, autonomous vehicles, and medical image analysis.")
]

result = chain.invoke({"input_documents": documents})
print(result["output_text"])


Artificial Intelligence is revolutionizing various industries globally by utilizing machine learning algorithms, enabling computers to learn from data and improve their performance over time without explicit programming. Key areas of AI include Natural Language Processing, which allows machines to understand and generate human language, powering applications such as chatbots, translation services, and sentiment analysis. Additionally, computer vision enables machines to interpret and understand visual information, driving technologies like facial recognition, autonomous vehicles, and medical image analysis. Together, these advancements enhance the capabilities of AI in real-world scenarios.




Each document chunk refines the answer.

---

### Internal Execution (Simplified)

```
answer = LLM(initial_prompt(chunk1))
for chunk in remaining_chunks:
    answer = LLM(refine_prompt(answer, chunk))
return answer
```

---

### When Refine Chain Is Best

* Long documents where order matters
* Reports, books, transcripts
* Progressive summarization
* Detailed synthesis tasks

---

### Limitations of Refine Chain

* ❌ Sequential (no parallelism)
* ❌ Higher latency
* ❌ More fragile to early mistakes
* ❌ Legacy abstraction

---

### Refine Chain vs Stuff Chain

| Aspect | Refine | Stuff |
|------|-------|
Scalability | High | Low |
Context retention | High | Low |
Latency | Higher | Low |

---

### Refine Chain vs LCEL (Modern)

Refine logic can be expressed using LCEL with:

* Stateful accumulation
* Custom reducers
* Streaming control

LCEL provides:

* Better observability
* Retry & fallback
* Custom refinement strategies

---

### Best Practices

* Use deterministic parameters
* Keep initial answer concise
* Limit number of chunks
* Monitor hallucinations
* Validate final output

---

### When NOT to Use Refine Chain

* Small inputs
* Real-time APIs
* Highly parallel workloads
* New projects (prefer LCEL)

---

### Interview-Ready Summary

> “A Refine Chain incrementally improves an answer by feeding each new document chunk along with the previous response back into the model. It preserves context better than MapReduce but has higher latency and is a legacy LangChain abstraction.”

---

### Rule of Thumb

* **Order matters → Refine**
* **Scale matters → MapReduce**
* **Small input → Stuff**
* **New systems → LCEL**